# APIs 101 (oDCM)

*The focus in this tutorial lies on pagination (i.e., looping through multiple pages), and parameters (i.e., modifying the response of an API call). We know you love dad jokes, so guess what? We're back with many more jokes, and you're going to learn how to save them all! Finally, we show you how to obtain user-level data from the Reddit!*

--- 

## Learning Objectives

* Send HTTP requests to a web API, and retrieve JSON responses
* Use parameters to modify the results of an API call
* Iterate over multiple pages of JSON responses 
* Extract and store results of an API request in lists and files

--- 

## Acknowledgements
This course draws on a variety of online resources which can be retrieved from the [course website](https://odcm.hannesdatta.com/#student-profile--prerequisites). 


--- 

## Support Needed?
For technical issues outside of scheduled classes, please check the [support section](https://odcm.hannesdatta.com/docs/course/support) on the course website.

---

## 1. Icanhazdajoke

### 1.1 Make an API request

[Icanhazdadjoke.com](https://icanhazdadjoke.com) is a simple web site that allows users of their API to receive (randomized) *dad jokes*. Yes, we know that sounds stupid, but we like that API for its simplicity, which is ideal when explaining to you more about APIs.

So, the code cell below calls the joke API, and the result of the API request displays a joke. 

__Let's try it out__

Run the cell a few times to notice that with each call, you see a new joke.


In [36]:
# request JSON output from icanhazdadjoke API
import requests
url = "https://icanhazdadjoke.com"
response = requests.get(url, headers={"Accept": "application/json"})
joke_request = response.json() 
print(joke_request)

{'id': 'WvsHlyszkyd', 'joke': 'Why did the man put his money in the freezer? He wanted cold hard cash!', 'status': 200}


### 1.2 Use parameters to modify the API results   

__Importance__

Probably you agree that dad jokes per se aren't that exciting. Wouldn't it be amazing to search for particular jokes instead?

APIs certainly provide the functionality to *customize* requests. That's where APIs make most of a difference! You have probably already modified the results of an API call a dozen times without even knowing it. For example, if you Google the word `cat`, the results page may look something like this:

<img src="https://raw.githubusercontent.com/hannesdatta/course-odcm/dev/content/docs/tutorials/apis101/images/google.png" width=60% align="left"  style="border: 1px solid black"/>


Note how the URL in the browser starts with [`google.com/search?q=cat...`](https://www.google.com/search?q=cat)? What happened here is that your search query was passed to the Google Search API, and hence returned the results of the search query `cat`. That search query is even already embedded in the link itself. Cool, right?

__Let's try it out__

So, rather than filling out the search box on the website of Icanhazdadjoke.com itself, you can also tweak it in the URL directly. Open your browser now at [https://icanhazdadjoke.com/search?term=cat](https://icanhazdadjoke.com/search?term=cat), and modify the `term` parameter to try a search for different jokes.


<img src="https://raw.githubusercontent.com/hannesdatta/course-odcm/dev/content/docs/tutorials/apis101/images/cat_jokes.gif" width=60% align="left"  style="border: 1px solid black"/>

With the idea of passing parameters to a website, we can update the `search_url` and include the `params` attribute, which contains a dictionary with parameters that further specify our request. Run the cell below to see cat jokes here in Jupyter Notebook.

In [37]:
import requests
search_url = "https://icanhazdadjoke.com/search"

response = requests.get(search_url, 
                        headers={"Accept": "application/json"}, 
                        params={"term": "cat"})
joke_request = response.json()
print(joke_request)

{'current_page': 1, 'limit': 20, 'next_page': 1, 'previous_page': 1, 'results': [{'id': '8UnrHe2T0g', 'joke': '‘Put the cat out’ … ‘I didn’t realize it was on fire'}, {'id': 'iGJeVKmWDlb', 'joke': 'My cat was just sick on the carpet, I don’t think it’s feline well.'}, {'id': 'daaUfibh', 'joke': 'Why was the big cat disqualified from the race? Because it was a cheetah.'}, {'id': '1wkqrcNCljb', 'joke': "Did you know that protons have mass? I didn't even know they were catholic."}, {'id': 'BQfaxsHBsrc', 'joke': 'What do you call a pile of cats?  A Meowtain.'}, {'id': 'O7haxA5Tfxc', 'joke': 'Where do cats write notes?\r\nScratch Paper!'}, {'id': 'TS0gFlqr4ob', 'joke': 'What do you call a group of disorganized cats? A cat-tastrophe.'}, {'id': '0wcFBQfiGBd', 'joke': 'Did you hear the joke about the wandering nun? She was a roman catholic.'}, {'id': 'AQn3wPKeqrc', 'joke': 'It was raining cats and dogs the other day. I almost stepped in a poodle.'}, {'id': '39Etc2orc', 'joke': 'Why did the man

The `joke_request` object now contains a list with all cat-related jokes (`joke_request['results']`), the search term (`cat`), and the total number of jokes (`10`).

#### Exercise 1
1. Change the search term parameter to `dog` and revisit `joke_request['results']`. How many dog jokes are there? 
2. Write a function `find_joke()` that takes a query as an input parameter and returns the number of jokes from the `icanhazdadjoke` search API (tip: use your answer to question 1 as a starting point!). 




In [ ]:
# your answer goes here!

#### Solutions

In [5]:
# Question 1 
search_url = "https://icanhazdadjoke.com/search"

response = requests.get(search_url, 
                        headers={"Accept": "application/json"}, 
                        params={"term": "dog"})
joke_request = response.json()
print(f"The number of dog jokes is: {joke_request['total_jokes']}")

The number of dog jokes is: 12


In [11]:
# Question 2
def find_jokes(term):
    search_url = "https://icanhazdadjoke.com/search"

    response = requests.get(search_url, 
                            headers={"Accept": "application/json"}, 
                            params={"term": term})
    joke_request = response.json()
    num_results = joke_request['total_jokes']
    return num_results

find_jokes("some-searchterm-you-would-like-to-try-out")

0

### 1.3 Pagination

__Importance__

Transferring data is costly - not strictly in a monetary sense, but in *time*. So - APIs are typically very greedy in returning data. Ideally, they only produce a very targeted data point that is needed for the user to see. On icanhazdadjoke.com, for example, that would be a few jokes at maximum. It saves the web site owner paying for bandwidth and guarantees that the site responds fast to user input (such as navigating the site or searching for jokes).

However, when using APIs for research purposes, we are frequently interested in obtaining *everything*. What's the use, for example, to get a book's most recent ten reviews, if there are hundreds of reviews written?

We think you see where we're going with this... 

__Let's try it out__

So, let's try to grab all of the 649 jokes currently available at Icanhazdadjoke.com. The API output, unfortunately, only shows the *first 20 jokes*. To retrieve the remaining 629 jokes, you need *pagination*. The API divides the data into smaller subsets that can be accessed on various pages, rather than returning all output at once. 

Let's retrieve the first batch of dad jokes (note, here we're searching for the `term` `""` - an empty string - which brings us to the entire set of jokes available via the API. In practice, searching for `""` is often blocked by APIs - simply because the site doesn't *want* you to extract a complete copy of their data. In that case, you'd have to become creative to obtain your seeds.


In [18]:
search_url = "https://icanhazdadjoke.com/search"

response = requests.get(search_url, 
                        headers={"Accept": "application/json"}, 
                        params={"term": ""})
joke_request = response.json()
joke_request['results'] = '' # let's remove all jokes, and only look at the other attributes in the JSON response
joke_request

{'current_page': 1,
 'limit': 20,
 'next_page': 2,
 'previous_page': 1,
 'results': '',
 'search_term': '',
 'status': 200,
 'total_jokes': 649,
 'total_pages': 33}

You notice that by default, each page contains 20 jokes (see `limit` in the JSON response above), where page 1 shows jokes 1 to 20, page 2 jokes 21 to 40, ..., and page 33 jokes 641 to 649. 

You can adjust the number of results on each page (max. 30) with the `limit` parameter (e.g., `params={"limit": 10}`). In practice, almost every API on the web limits the results of an API call (`100` is also a common cap).

In the example below, we set `limit` equal to `10`, `20`, and `30`, and see how it affects the number of total pages (`total_pages`) on which jokes are listed. 

In [38]:
for limit in range(10, 31, 10):  # note that range(a, b) runs from a to b-1; so the last value is exclusive (so from 10 to 30 with steps of 10)
    response = requests.get(search_url, 
                            headers={"Accept": "application/json"}, 
                            params={"term": "", 
                                   "limit": limit})
    joke_request = response.json()
    print(f"Limit {limit} gives {joke_request['total_pages']} pages")

Limit 10 gives 65 pages
Limit 20 gives 33 pages
Limit 30 gives 22 pages


As expected, we find that the higher the limit, the more results fit on a single page, and thus the *lower the number of pages* to loop through.

--- 
#### Exercise 2

In addition to the `limit` parameter, you can specify the current page number with the `page` parameter (e.g., `params={"term": "", "page": 2}`. See the example in the next cell:

In [25]:
response = requests.get(search_url, 
                            headers={"Accept": "application/json"}, 
                            params={"term": "", 
                                   "limit": 5,
                                   "page": 2})
response.json()

{'current_page': 2,
 'limit': 5,
 'next_page': 3,
 'previous_page': 1,
 'results': [{'id': '0LuXvkq4Muc',
   'joke': "I knew I shouldn't steal a mixer from work, but it was a whisk I was willing to take."},
  {'id': '0ga2EdN7prc',
   'joke': 'How come the stadium got hot after the game? Because all of the fans left.'},
  {'id': '0oO71TSv4Ed',
   'joke': 'Why was it called the dark ages? Because of all the knights. '},
  {'id': '0oz51ozk3ob', 'joke': 'A steak pun is a rare medium well done.'},
  {'id': '0ozAXv4Mmjb',
   'joke': 'Why did the tomato blush? Because it saw the salad dressing.'}],
 'search_term': '',
 'status': 200,
 'total_jokes': 649,
 'total_pages': 130}

Adapt the function `find_joke()` (see question 2 of exercise 1), such that it loops over *all available pages*, and stores the ids and jokes in a list. You can leave the `limit` parameter at its default value (20). Make sure that your function also works when you pass it a search `term`. 

Tip: To determine how many pages you need to loop through, you can use the `total_pages` field (e.g., there are only ten cat jokes, so in that case, 1 page would suffice).

In [ ]:
# your answer goes here!

#### Solutions

In [33]:
def find_jokes(term):
    search_url = "https://icanhazdadjoke.com/search"
    page = 1
    jokes = []

    while True:  # alternatively you can also use a for-loop that goes from page 1 to total_results / 20 (rounded up)
        response = requests.get(search_url, 
                                headers={"Accept": "application/json"}, 
                                params={"term": term,  # optionally you can add "limit": 20 but that's already the default so it doesn't change anything
                                        "page": page})
        joke_request = response.json()
        jokes.extend(joke_request['results'])
        if joke_request['current_page'] <= joke_request['total_pages']:
            page += 1
        else: 
            return jokes

output = find_jokes("cat") # try running it with "", too!

In [34]:
print(f"You've collected {len(output)} jokes")

You've collected 10 jokes


### 1.4 Wrap-up

To sum up, we have seen how *parameters* can be a powerful tool when working with APIs. They allow you to tailor your request to be more specific or loop through multiple pages. 

In the API documentation, you typically find more information about the available parameters and the values they can take on. For example, the `icanhazdadjoke` [documentation](https://icanhazdadjoke.com/api) includes a section on the `/search` endpoint and the accepted parameters (`page`, `limit`, `term`). These parameters, however, differ from one API to another. So it's crucial to study each web service's API documentation carefully.


--- 
## 2. Reddit

### 2.1 Subreddits

[Reddit](https://reddit.com) is a widespread American social news aggregation and discussion site. The service uses an API to generate the website's content and grants public access to the API.

In this tutorial, we zoom in on "subreddits", which are niche communities centered around a particular topic. Users can nearly post anything in these subreddits, and you'd be surprised to find out what people are talking about. For example, see below for a screenshot of the [subreddit on Science](https://www.reddit.com/r/Science).

<img src="https://raw.githubusercontent.com/hannesdatta/course-odcm/dev/content/docs/tutorials/apis101/images/reddit_science.png" width=60% align="left"  style="border: 1px solid black"/>


__Let's try it out__

Subreddits all start with `reddit.com/r/...`. Here are a few examples:

- [askreddit](https://www.reddit.com/r/AskReddit), 
- [aww](https://www.reddit.com/r/aww/), 
- [gifs](https://www.reddit.com/r/gifs/), 
- [showerthoughts](https://www.reddit.com/r/Showerthoughts), 
- [lifehacks](https://www.reddit.com/r/lifehacks), 
- [getmotivated](https://www.reddit.com/r/GetMotivated), 
- [moviedetails](https://www.reddit.com/r/MovieDetails), 
- [todayilearned](https://www.reddit.com/r/todayilearned/), 
- [foodporn](https://www.reddit.com/r/FoodPorn/). 

Take your time to browse through some of the subreddits, and get familiar with the structure of the pages.

After a while, you'd probably notice that subreddits are hosted by moderators, who monitor whether the posts adhere to a set of (informal) rules.

<img src="https://raw.githubusercontent.com/hannesdatta/course-odcm/dev/content/docs/tutorials/apis101/images/reddit_moderators.png" width=60% align="left"  style="border: 1px solid black"/>

For example, links to papers you share in [`r/science`](https://www.reddit.com/r/science/) must be less than 6 months old. 

Other users can join a subreddit so that they receive updates about new posts and comments.




#### Exercise 3
Consult the [`marketing`](https://www.reddit.com/r/marketing/hot/) subreddit and answer the following questions: 
1. For your thesis, you need to collect survey responses. Are you allowed to share a link to your survey in this subreddit? Please elaborate on how you came to this conclusion. 
2. You post a link (and therefore run the risk of being reported by one of the moderators). How many moderators take care of managing this subreddit?
3. Like other social media platforms, you can navigate towards Reddit's user-profiles and learn more about these persons. Inspect the profile of one of the moderators of the marketing subreddit, [`sixwaystop313`](https://www.reddit.com/user/sixwaystop313), and describe in your own words what types of information you can gather from this user. How is the feed organized?


In [ ]:
# your answer goes here!

#### Solutions
1. No, the subreddit rules prescribe users not to post surveys and homework assignments (right sidebar).
2. `r/marketing` is moderated by ten users (of which 1 AutoModerator)
3. On a user page, you find the bio, trophies, communities the user moderates, connected accounts, and most importantly: all user's posts and comments.

---


### 2.2 API headers  

**Importance**  

Let's now obtain some of the moderators' data we've just seen via the Reddit API. 

To request data from the Reddit API, we need to include `headers` in our request. HTTP headers are a vital part of any API request, containing *meta-data associated with the request* (e.g., type of browser, language, expected data format, etc.). 

**Let's try it out**  

Below we request the moderators' page of the [`marketing`]() subreddit that includes such a header. We make our first request to the Reddit API and parse the output in the upcoming exercise!


In [1]:
import requests
url = 'https://www.reddit.com/r/marketing/about/moderators/.json'

headers = {'authority': 'www.reddit.com', 'cache-control': 'max-age=0', 'upgrade-insecure-requests': '1', 'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36', 'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9', 'sec-fetch-site': 'same-origin', 'sec-fetch-mode': 'navigate', 'sec-fetch-user': '?1', 'sec-fetch-dest': 'document', 'accept-language': 'en-GB,en;q=0.9'}
response = requests.get(url, headers=headers)
json_response = response.json()

#### Exercise 4
1. First, take a look at the `json_response` object. Then, leave out the `headers` parameter in your request (so it becomes `requests.get(url)` instead), rerun the cell, and inspect the `json_response` another time. Are there any differences? 
2. Write a for-loop that prints the moderator `name` of the `marketing` subreddit. Every subreddit includes a bot moderator (`AutoModerator`), which should *not* be included.
3. Convert your code from the previous exercise into a function `get_mods()` that takes a `subreddit` as input and returns a list of moderators' names. Test your function for the `science` subreddit. How many moderators does it have?


In [ ]:
# your answer goes here!

#### Solutions
1. Without the `headers` parameter, the API returns an error code (429). Headers are frequently used to track who is using the API. The user of the "anonymous header" has pushed the boundaries too much!


In [121]:
# Question 2 
# don't forget to run the request object with headers again!
for item in json_response['data']['children']: 
    moderator_name = item['name']
    if moderator_name != 'AutoModerator': 
        print(moderator_name)

dpatrick86
v022450781
r0nin
Gustomaximus
everythingswan
sixwaystop313
shampine
JonODonovan
AptSeagull


In [37]:
# Question 3
def get_mods(subreddit):
    moderator_names = []
    response = response = requests.get(f'https://www.reddit.com/r/{subreddit}/about/moderators/.json', headers=headers)
    json_response = response.json()
    for item in json_response['data']['children']:
        moderator_name = item['name']
        if moderator_name != 'AutoModerator':
            moderator_names.append(moderator_name)
    return moderator_names
    
science_moderators = get_mods('science')
print(f"The science subreddit is moderated by {len(science_moderators)} users")

The science subreddit is moderated by 1540 users


---
### 2.3 Moderator pages

In addition to subreddits (`r/...`) and moderator pages (`.../about/moderators`), Reddit users have their own profile page. Let's have another look at the marketing moderator [profile](https://www.reddit.com/user/sixwaystop313) we saw before. Each of the `children` in the `data` is characterized by a type (e.g., `t1` = comment, `t3` = post; for details see [API documentation](https://redditclient.readthedocs.io/en/latest/reference/)), subreddit, timestamp, number of comments, upvotes, downvotes, and many others. 

__Let's try it out__

Run the API call below, and browse the results.

In [5]:
mod = "sixwaystop313"
response = requests.get(f'https://www.reddit.com/user/{mod}.json', headers=headers)
json_response = response.json()
json_response

{'kind': 'Listing',
 'data': {'modhash': '',
  'dist': 25,
  'children': [{'kind': 't3',
    'data': {'approved_at_utc': None,
     'subreddit': 'Jeep',
     'selftext': '',
     'author_fullname': 't2_3pmgd',
     'saved': False,
     'mod_reason_title': None,
     'gilded': 0,
     'clicked': False,
     'title': 'Jeep will air "The Middle", 2-minute short film in it\'s entirety during 2021 Super Bowl',
     'link_flair_richtext': [],
     'subreddit_name_prefixed': 'r/Jeep',
     'hidden': False,
     'pwls': 6,
     'link_flair_css_class': None,
     'downs': 0,
     'thumbnail_height': 105,
     'top_awarded_type': None,
     'hide_score': False,
     'name': 't3_legybw',
     'quarantine': False,
     'link_flair_text_color': 'dark',
     'upvote_ratio': 0.83,
     'author_flair_background_color': None,
     'subreddit_type': 'public',
     'ups': 20,
     'total_awards_received': 0,
     'media_embed': {'content': '&lt;iframe width="356" height="200" src="https://www.youtube.com

That's a whole lot of output, which is difficult to go through. So let's copy it to a [JSON viewer](https://jsonviewer.stack.hu). Before we can do that, we have to replace the Pythonic `None`, `True` and `False` by strings (JSON viewer throws an error otherwise).

In [6]:
import json
json_response = json.loads(response.text.replace('null', '"None"').replace('true','"True"').replace('false','"False"'))
json_response

{'kind': 'Listing',
 'data': {'modhash': '',
  'dist': 25,
  'children': [{'kind': 't3',
    'data': {'approved_at_utc': 'None',
     'subreddit': 'Jeep',
     'selftext': '',
     'author_fullname': 't2_3pmgd',
     'saved': 'False',
     'mod_reason_title': 'None',
     'gilded': 0,
     'clicked': 'False',
     'title': 'Jeep will air "The Middle", 2-minute short film in it\'s entirety during 2021 Super Bowl',
     'link_flair_richtext': [],
     'subreddit_name_prefixed': 'r/Jeep',
     'hidden': 'False',
     'pwls': 6,
     'link_flair_css_class': 'None',
     'downs': 0,
     'thumbnail_height': 105,
     'top_awarded_type': 'None',
     'hide_score': 'False',
     'name': 't3_legybw',
     'quarantine': 'False',
     'link_flair_text_color': 'dark',
     'upvote_ratio': 0.83,
     'author_flair_background_color': 'None',
     'subreddit_type': 'public',
     'ups': 20,
     'total_awards_received': 0,
     'media_embed': {'content': '&lt;iframe width="356" height="200" src="htt

#### Exercise 5
1. The `json_response` object contains both comments and posts ordered chronologically (exactly as they appear on the profile page). Pick a comment (`kind`: `'t1'`) of the author and store the text of the comment in a variable called `comment_text`. 
2. What happens to `comments_text` once the author publishes another post? 
3. How many objects are stored in `json_response['data']['children']`? What does that mean? 

In [ ]:
# your answer goes here!

#### Solutions
1. At the moment of creating this solutions file, the 1st item in the list is a comment which we extract as follows:
`comment_text = json_response['data']['children'][0]['data']`. In your case, it may be 2nd (or 3rd, 4th, ... item), however, provided that all other items in the lists are posts. For that reason, the counter after `[0]` may deviate from time to time. 


In [ ]:
# if this solution throws a "KeyError: body" error it means the most recent JSON object is not a comment of kind t1 (so change the 0 for 1, 2, ... until it runs) - see question 2
comment_text = json_response['data']['children'][0]['data']['body']
comment_text

2. Since the list items are ordered chronologically, new items are appended at the *beginning* of the list and thus push existing items to the "right" (i.e., index 0 becomes index 1, etc.). Suppose that the author publishes another post, then index `[0]` would no longer contain a comment. Post items have been structured differently from comment items, which could potentially break your script once you try to parse non-existing items. For example, posts do not have a `['body']` element that stores the comment text.

3. The object comprises 25 items, which implies that only the 25 most recent comments and posts are shown. Thus, we need to apply pagination to obtain historical records.

In [82]:
len(json_response['data']['children'])

25

### 2.4 Pagination

__Importance__

As you've just noticed, the API only returns a subset of all records (every time you scroll to the bottom of the page, it pulls in new data - ordered chronologically). After all, it would take ages to show all data for a user that has been active on Reddit since 2009! 

__Let's try it out__

Similar to `icanhazdadjoke`, we apply pagination to tell the API which part of the data to return. The difference, however, is that it's not a number (like `"page": 2`) but a string of characters that can only be obtained from the previous request (i.e., we cannot derive what the next key will be from a pattern, like page 2, 3, ..., etc.). The request we already made contains this "secret" key in the attribute `after`:


In [28]:
json_response['data']['after']

't1_gjp36fk'

Next, we attach this key to our request with the `after` parameter to obtain the next subset of items and assign the responses to a variable called `json_response_after`: 

In [29]:
after = json_response['data']['after']
url = f'https://www.reddit.com/user/{mod}.json'
response = requests.get(url, 
                        headers=headers, 
                        params={"after": after})
json_response_after = response.json()
json_response_after

{'kind': 'Listing',
 'data': {'modhash': '',
  'dist': 25,
  'children': [{'kind': 't1',
    'data': {'total_awards_received': 0,
     'approved_at_utc': None,
     'comment_type': None,
     'awarders': [],
     'mod_reason_by': None,
     'banned_by': None,
     'author_flair_type': 'text',
     'removal_reason': None,
     'link_id': 't3_kyng2a',
     'author_flair_template_id': None,
     'likes': None,
     'replies': '',
     'user_reports': [],
     'saved': False,
     'id': 'gjhnl61',
     'banned_at_utc': None,
     'mod_reason_title': None,
     'gilded': 0,
     'archived': False,
     'no_follow': True,
     'author': 'sixwaystop313',
     'num_comments': 32,
     'edited': False,
     'can_mod_post': False,
     'created_utc': 1610822716.0,
     'send_replies': True,
     'parent_id': 't1_gjhfwl9',
     'score': 1,
     'author_fullname': 't2_3pmgd',
     'over_18': False,
     'treatment_tags': [],
     'approved_by': None,
     'mod_note': None,
     'all_awardings': []

At the point of writing this tutorial (when you're doing this tutorial it's likely different!), the last item in `json_respose` is the following post (`Detroit's Brewing Heritage' on tap at Historical Museum`): 

<img src="https://raw.githubusercontent.com/hannesdatta/course-odcm/dev/content/docs/tutorials/apis101/images/json_response.png" width=60% align="left"/>

The first and second items in `json_response_after` are the two comments below that ("Shame on ... us back." and "Are you ... comment /u/ehchip"). In other words, where one object ends, another begins. We apply this concept to loop over the first ten pages. At each iteration, we store the `after` attribute, which we use as a parameter in the follow-up request.

In [32]:
after = None
item_type = []

for counter in range(10): 
    url = f'https://www.reddit.com/user/{mod}.json'
    print('processing ' + url + ' with after parameter: ' + str(after))
    response = requests.get(url, 
                            headers=headers, 
                            params={"after": after})
    json_response = response.json()
    after = json_response['data']['after'] 

    # loop over all items in a request
    for item in json_response['data']['children']:
        item_type.append(item['kind'])

# Let's view the item types: 
item_type

processing https://www.reddit.com/user/sixwaystop313.json with after parameter: None
processing https://www.reddit.com/user/sixwaystop313.json with after parameter: t1_gjp36fk
processing https://www.reddit.com/user/sixwaystop313.json with after parameter: t1_ghktae1
processing https://www.reddit.com/user/sixwaystop313.json with after parameter: t1_gg7d64v
processing https://www.reddit.com/user/sixwaystop313.json with after parameter: t1_ge32rap
processing https://www.reddit.com/user/sixwaystop313.json with after parameter: t1_gcgn52k
processing https://www.reddit.com/user/sixwaystop313.json with after parameter: t1_gar0g4m
processing https://www.reddit.com/user/sixwaystop313.json with after parameter: t1_g8iw603
processing https://www.reddit.com/user/sixwaystop313.json with after parameter: t1_g5i98ga
processing https://www.reddit.com/user/sixwaystop313.json with after parameter: t3_ik5rkp


['t3',
 't3',
 't3',
 't3',
 't1',
 't1',
 't1',
 't1',
 't3',
 't1',
 't1',
 't1',
 't1',
 't1',
 't1',
 't1',
 't1',
 't1',
 't1',
 't3',
 't1',
 't3',
 't3',
 't1',
 't1',
 't1',
 't3',
 't3',
 't1',
 't1',
 't1',
 't3',
 't1',
 't1',
 't3',
 't1',
 't1',
 't1',
 't1',
 't1',
 't1',
 't3',
 't1',
 't3',
 't1',
 't1',
 't1',
 't3',
 't3',
 't1',
 't1',
 't1',
 't1',
 't1',
 't1',
 't1',
 't1',
 't3',
 't1',
 't1',
 't3',
 't1',
 't1',
 't1',
 't3',
 't1',
 't1',
 't1',
 't1',
 't1',
 't1',
 't1',
 't3',
 't1',
 't1',
 't3',
 't1',
 't1',
 't1',
 't1',
 't3',
 't1',
 't3',
 't3',
 't3',
 't1',
 't1',
 't1',
 't1',
 't3',
 't1',
 't3',
 't3',
 't3',
 't1',
 't1',
 't3',
 't1',
 't1',
 't1',
 't1',
 't1',
 't3',
 't1',
 't1',
 't3',
 't1',
 't1',
 't1',
 't1',
 't1',
 't3',
 't1',
 't3',
 't1',
 't1',
 't1',
 't3',
 't3',
 't3',
 't1',
 't1',
 't3',
 't3',
 't1',
 't1',
 't3',
 't1',
 't3',
 't3',
 't1',
 't1',
 't3',
 't1',
 't1',
 't3',
 't1',
 't1',
 't1',
 't1',
 't1',
 't1',
 't1',

#### Exercise 6
1. Why do we define `after = None` at the top of the file? Can we leave it out? 
2. Without looking at the list's length, how many items do you expect in `item_type`? 
3. Of all items in `item_type`, calculate the percentage of posts (`t3`) and comments (`t1`). What does this tell you? 
4. Convert the code snippet above into a function `reddit_activity()` that takes a `username`, `attribute`, and `num_pages` as inputs and returns the attribute for the given user. For example, `reddit_activity("sixwaystop313", "subreddit_name_prefixed", 40)` should return a list of the subreddits in which the user has posted or commented across the 1000 most recent items. 
5. Use the function written in 3 to assess whether the moderator has actively contributed to the `r/marketing` subreddit recently?

In [ ]:
# your answer goes here!

#### Solutions 
1. In our first request we don't know the value of `after` yet. It is important, however, to include this line because otherwise the `after` value in `params={}` is undefined. 
2. We expect the list to have a size of 10 (number of requests) * 25 (number of items per request) = 250. 

In [33]:
# Question 3
def item_frequency(items, item_filter):
    total_items = len(items)
    item_filter_count = items.count(item_filter)
    return item_filter_count / total_items * 100
            
perc_posts = item_frequency(item_type, 't1')
perc_comments = item_frequency(item_type, 't3')

print(f"The percentage of posts and comments is {perc_posts}% and {perc_comments}%, respectively")
# Thus, based on this subset of data, the author is more likely to start a new post than to comment on others' posts

The percentage of posts and comments is 69.19999999999999% and 30.8%, respectively


In [40]:
# Question 4
def reddit_activity(username, attribute, num_pages):
    after = None
    activity = []

    for counter in range(num_pages): 
        url = f'https://www.reddit.com/user/{username}.json'
        print('processing ' + url + ' with after parameter: ' + str(after))
        response = requests.get(url, 
                                headers=headers, 
                                params={"after": after})
        json_response = response.json()
        after = json_response['data']['after']

        # loop over all items in a request
        for item in json_response['data']['children']:
            activity.append(item['data'][attribute])
    return activity

reddit_data = reddit_activity("sixwaystop313", "subreddit_name_prefixed", 40)

processing https://www.reddit.com/user/sixwaystop313.json with after parameter: None
processing https://www.reddit.com/user/sixwaystop313.json with after parameter: t1_gjp36fk
processing https://www.reddit.com/user/sixwaystop313.json with after parameter: t1_ghktae1
processing https://www.reddit.com/user/sixwaystop313.json with after parameter: t1_gg7d64v
processing https://www.reddit.com/user/sixwaystop313.json with after parameter: t1_ge32rap
processing https://www.reddit.com/user/sixwaystop313.json with after parameter: t1_gcgn52k
processing https://www.reddit.com/user/sixwaystop313.json with after parameter: t1_gar0g4m
processing https://www.reddit.com/user/sixwaystop313.json with after parameter: t1_g8iw603
processing https://www.reddit.com/user/sixwaystop313.json with after parameter: t1_g5i98ga
processing https://www.reddit.com/user/sixwaystop313.json with after parameter: t3_ik5rkp
processing https://www.reddit.com/user/sixwaystop313.json with after parameter: t1_g1ztn64
proces

In [107]:
reddit_data

['r/Detroit',
 'r/Detroit',
 'r/Detroit',
 'r/Detroit',
 'r/Detroit',
 'r/redditrequest',
 'r/Detroit',
 'r/Detroit',
 'r/Detroit',
 'r/Detroit',
 'r/ActiveMeasures',
 'r/Detroit',
 'r/Detroit',
 'r/Detroit',
 'r/Detroit',
 'r/Detroit',
 'r/Detroit',
 'r/Detroit',
 'r/Detroit',
 'r/Detroit',
 'r/Detroit',
 'r/republicans',
 'r/Detroit',
 'r/Detroit',
 'r/michigandemocrats',
 'r/Detroit',
 'r/woahdude',
 'r/Detroit',
 'r/Detroit',
 'r/Detroit',
 'r/Detroit',
 'r/Detroit',
 'r/Detroit',
 'r/Detroit',
 'r/Detroit',
 'r/Detroit',
 'r/Detroit',
 'r/Detroit',
 'r/Detroit',
 'r/Detroit',
 'r/michigandemocrats',
 'r/Detroit',
 'r/Detroit',
 'r/Detroit',
 'r/Detroit',
 'r/Detroit',
 'r/michigandemocrats',
 'r/Detroit',
 'r/Detroit',
 'r/Detroit',
 'r/Michigan',
 'r/Michigan',
 'r/michigandemocrats',
 'r/Michigan',
 'r/michigandemocrats',
 'r/Detroit',
 'r/Detroit',
 'r/michigandemocrats',
 'r/michigandemocrats',
 'r/politics',
 'r/Detroit',
 'r/Detroit',
 'r/Detroit',
 'r/Detroit',
 'r/michigan

In [108]:
print(f"The percentage of posts and comments in the marketing subreddit is {item_frequency(reddit_data, 'r/marketing')}%")
# Thus, the moderator has not actively contributed to the marketing subreddit recently

The percentage of posts and comments in the marketing subreddit is 0.0%


---
### 2.5 Time Conversion

__Importance__

When retrieving data from an API - in particular *user-level data*, not only the content of a post or comment matters, but also when the comment or post was written. In other words, we seek to extract the date and time (timestamps) from a users' comments and posts on Reddit.

__Let's try it out__

Run the cell below, to extract the timestamp from `sixwaystop313`'s most recent activity on Reddit.


In [113]:
url = 'https://www.reddit.com/user/sixwaystop313.json'
response = requests.get(url, headers=headers)
response.json()['data']['children'][0]['data']['created_utc']

1608526471.0

Hm... is *that* really a timestamp?

Well... computers handle time differently than humans. And what programmers somewhat converged to is that timestamps are best measured in *the number of seconds passed since 1 January, 1970*. With the use of the `time` library, we can easily convert it into a readable date and time:

In [184]:
import time 
time_example = response.json()['data']['children'][0]['data']['created_utc']
time_converted = time.gmtime(time_example)
print(time_converted)

time.struct_time(tm_year=2020, tm_mon=12, tm_mday=21, tm_hour=4, tm_min=54, tm_sec=31, tm_wday=0, tm_yday=356, tm_isdst=0)


From `time_converted` you can extract the day, month, and year separately:

In [115]:
print(f"The day is: {time_converted.tm_mday}")
print(f"The month is: {time_converted.tm_mon}")
print(f"The year is: {time_converted.tm_year}")

The day is: 21
The month is: 12
The year is: 2020


Or together, like this (characters that start with `%` have a special meaning, the `-` in  between these characters are literally the dashes you see in the output): 

In [179]:
print(time.strftime("%d-%m-%Y", time_converted))  
# %d = day
# %m = month
# %Y = year (4 digits) and %y = year (2 digits)

09-06-2020


--- 
#### Exercise 7 
1. In a similar way, you can convert the UTC time into an hour (`%H`) and minute (`%M`). Transform `time_example` into a readable time. The output should be `06:17`. 
2. Suppose we want to analyze the Reddit use of `sixwaystop313` throughout the day. More specifically, we want to know during what hours the user is most active on the platform. 
  * Use the function `reddit_activity()` you wrote earlier to pull in the UTC timestamps (set `num_items` to `10`). 
  * Extract the hour from these timestamps. 
  * Determine the top 3 hours the user is most active on Reddit. You can assume that the total number of posts and comments is a reasonable proxy for time spend on the platform. 

In [ ]:
# your answer goes here!

#### Solutions

In [117]:
# Question 1 
print(time.strftime("%H:%M", time_converted))

04:54


In [119]:
# Question 2
time_data = reddit_activity("sixwaystop313", "created_utc", 10)
hours = []

for timestamp in time_data: 
    time_converted = time.gmtime(timestamp)
    hours.append(time_converted.tm_hour)
    
for hour in range(24):
    print(f"Hour {hour}: {hours.count(hour)} items")
    
# Check out which hours are listed most to find the answer!

processing https://www.reddit.com/user/sixwaystop313.json with after parameter: None
processing https://www.reddit.com/user/sixwaystop313.json with after parameter: t3_k84pv3
processing https://www.reddit.com/user/sixwaystop313.json with after parameter: t1_gda1tjr
processing https://www.reddit.com/user/sixwaystop313.json with after parameter: t1_gbo3alg
processing https://www.reddit.com/user/sixwaystop313.json with after parameter: t1_g9o27zu
processing https://www.reddit.com/user/sixwaystop313.json with after parameter: t1_g76o1xl
processing https://www.reddit.com/user/sixwaystop313.json with after parameter: t1_g4g4u0d
processing https://www.reddit.com/user/sixwaystop313.json with after parameter: t1_g2y1hl9
processing https://www.reddit.com/user/sixwaystop313.json with after parameter: t1_g0qyp5m
processing https://www.reddit.com/user/sixwaystop313.json with after parameter: t3_hpd9f6
Hour 0: 16 items
Hour 1: 20 items
Hour 2: 15 items
Hour 3: 22 items
Hour 4: 27 items
Hour 5: 7 ite

---

### 2.6 Building an API extraction module

__Importance__

Up to now, we've written functions that in itself carry out separate tasks: one for obtaining the names of moderators of a particular subreddit, and one to obtain particular attributes of that user.

However, when we use APIs for research, we are not so much interested in the results of "single-shot" API requests, but we would like to obtain a *copy* of the entire data, so that we can analyze it later.

So, the purpose of this section is to "stitch" together the individual pieces. For now, we assume that we are interested in studying the Reddit behavior of any of the moderators interested in marketing. Hence, we need to...

- obtain a list of all moderators of the marketing subreddit, and
- store all of their posts and comments in a dataset.

__Let's try it out__

Let's first recall again how the `get_mods()` function from above worked. Let's assume we're interested in the `marketing` subreddit for now.

In [38]:
moderators = get_mods('marketing')
moderators

['dpatrick86',
 'v022450781',
 'r0nin',
 'Gustomaximus',
 'everythingswan',
 'sixwaystop313',
 'shampine',
 'JonODonovan',
 'AptSeagull']

We also have written our `reddit_activity` function, which we could call now using the (for prototypical purposes) first moderator.

In [41]:
reddit_activity(moderators[0], "created_utc", 3)

processing https://www.reddit.com/user/dpatrick86.json with after parameter: None
processing https://www.reddit.com/user/dpatrick86.json with after parameter: t1_emyw3bq
processing https://www.reddit.com/user/dpatrick86.json with after parameter: t3_9kh4ep


[1611261276.0,
 1609876550.0,
 1606926205.0,
 1606858947.0,
 1605216860.0,
 1598754652.0,
 1598458424.0,
 1593915259.0,
 1593203490.0,
 1592747969.0,
 1592674454.0,
 1591498238.0,
 1591336484.0,
 1591025573.0,
 1586983429.0,
 1584325730.0,
 1584322787.0,
 1580278832.0,
 1580244954.0,
 1579628566.0,
 1578518013.0,
 1572462158.0,
 1568313656.0,
 1559331210.0,
 1557445244.0,
 1557423244.0,
 1556746903.0,
 1556743590.0,
 1556743140.0,
 1555265297.0,
 1554224628.0,
 1551977602.0,
 1551542834.0,
 1551491938.0,
 1551468526.0,
 1551451509.0,
 1551451441.0,
 1550516976.0,
 1548797731.0,
 1548781045.0,
 1548775975.0,
 1547827464.0,
 1547827438.0,
 1539958719.0,
 1539958439.0,
 1539957347.0,
 1539957272.0,
 1538959768.0,
 1538947454.0,
 1538406384.0,
 1536251421.0,
 1535663436.0,
 1535661828.0,
 1533420165.0,
 1533010685.0,
 1531261097.0,
 1531260987.0,
 1531260795.0,
 1531193120.0,
 1528423927.0,
 1528400022.0,
 1528389840.0,
 1527141178.0,
 1525120282.0,
 1524880894.0,
 1523650784.0,
 152364485

The code above produced a list of timestamps of when that user was active. The function right now is not particularly useful yet, because it doesn't return multiple fields (but only one). So, let's first extend the reddit_activity function to...
- take on a `list` of attributes to extract, rather than just one
- to store the results in a dictionary, rather than a list (easier for writing to CSV later), and
- to always store the type of content (`kind`, e.g., post or comment).


In [42]:
def reddit_activity_updated(username, attributes, num_pages):
    after = None
    activity = []

    for counter in range(num_pages): 
        url = f'https://www.reddit.com/user/{username}.json'
        print('processing ' + url + ' with after parameter: ' + str(after))
        response = requests.get(url, 
                                headers=headers, 
                                params={"after": after})
        json_response = response.json()
        after = json_response['data']['after']

        # loop over all items in a request
        for item in json_response['data']['children']:
            tmp = {}
            tmp['kind'] = item['kind']
            for attribute in attributes:
                try:
                    tmp[attribute] = item['data'][attribute]
                except:
                    0 # do nothing
            activity.append(tmp)
    return activity

reddit_data = reddit_activity_updated(moderators[0], ["created_utc", "subreddit_name_prefixed", "body"], 10)

processing https://www.reddit.com/user/dpatrick86.json with after parameter: None
processing https://www.reddit.com/user/dpatrick86.json with after parameter: t1_emyw3bq
processing https://www.reddit.com/user/dpatrick86.json with after parameter: t3_9kh4ep
processing https://www.reddit.com/user/dpatrick86.json with after parameter: t1_dr8udl2
processing https://www.reddit.com/user/dpatrick86.json with after parameter: t1_dmp4kas
processing https://www.reddit.com/user/dpatrick86.json with after parameter: t1_div1xez
processing https://www.reddit.com/user/dpatrick86.json with after parameter: t1_dfbo1pf
processing https://www.reddit.com/user/dpatrick86.json with after parameter: t1_d5doj15
processing https://www.reddit.com/user/dpatrick86.json with after parameter: t1_cxfwgmo
processing https://www.reddit.com/user/dpatrick86.json with after parameter: t1_cn3hqrd


In [43]:
reddit_data

[{'kind': 't1',
  'created_utc': 1611261276.0,
  'subreddit_name_prefixed': 'r/coldshowers',
  'body': 'Fun fact... Norepinephrine is also the mechanism by which cold browns fat. When you adapt to the cold, one of the key adaptations is that browning and you owe it to norepinephrine.'},
 {'kind': 't1',
  'created_utc': 1609876550.0,
  'subreddit_name_prefixed': 'r/StandUpComedy',
  'body': 'This was great.'},
 {'kind': 't3',
  'created_utc': 1606926205.0,
  'subreddit_name_prefixed': 'r/PlantBasedDiet'},
 {'kind': 't3',
  'created_utc': 1606858947.0,
  'subreddit_name_prefixed': 'r/vegan'},
 {'kind': 't1',
  'created_utc': 1605216860.0,
  'subreddit_name_prefixed': 'r/coldshowers',
  'body': "How many buckets of ice is that? What's your setup like in this picture?"},
 {'kind': 't1',
  'created_utc': 1598754652.0,
  'subreddit_name_prefixed': 'r/Coronavirus',
  'body': 'Add only what you want by confirming on a diff-by-diff basis...\n`git add -p`'},
 {'kind': 't3',
  'created_utc': 1598

Observe now that the updated `reddit_activity_updated()` function returns multiple fields. The function also checks whether a field is actually *part* of the data (e.g., there is never a `body` for type `t3` (post), but only for `t1` (comments)

__Exercise 8__

1. Extend the `reddit_activity_updated()` function, so that it also stores a users' user name in the results.

2. Write a loop that calls `reddit_activity_updated()` on all moderators of the subreddit `marketing`, and stores the result in a (long) list of dictionaries. Limit yourself to the first 10 pages per user (to save time doing this exercise).

In [ ]:
# your answer goes here!

__Solutions__

*Question 1*

In [44]:
def reddit_activity_updated(username, attributes, num_pages):
    after = None
    activity = []

    for counter in range(num_pages): 
        url = f'https://www.reddit.com/user/{username}.json'
        print('processing ' + url + ' with after parameter: ' + str(after))
        response = requests.get(url, 
                                headers=headers, 
                                params={"after": after})
        json_response = response.json()
        after = json_response['data']['after']

        # loop over all items in a request
        for item in json_response['data']['children']:
            tmp = {}
            tmp['username'] = username # <-- SOLUTION TO QUESTION 1 IS HERE
            tmp['kind'] = item['kind']
            for attribute in attributes:
                try:
                    tmp[attribute] = item['data'][attribute]
                except:
                    0 # do nothing
            activity.append(tmp)
    return activity

reddit_data = reddit_activity_updated(moderators[0], ["created_utc", "subreddit_name_prefixed", "body"], 10)

processing https://www.reddit.com/user/dpatrick86.json with after parameter: None
processing https://www.reddit.com/user/dpatrick86.json with after parameter: t1_emyw3bq
processing https://www.reddit.com/user/dpatrick86.json with after parameter: t3_9kh4ep
processing https://www.reddit.com/user/dpatrick86.json with after parameter: t1_dr8udl2
processing https://www.reddit.com/user/dpatrick86.json with after parameter: t1_dmp4kas
processing https://www.reddit.com/user/dpatrick86.json with after parameter: t1_div1xez
processing https://www.reddit.com/user/dpatrick86.json with after parameter: t1_dfbo1pf
processing https://www.reddit.com/user/dpatrick86.json with after parameter: t1_d5doj15
processing https://www.reddit.com/user/dpatrick86.json with after parameter: t1_cxfwgmo
processing https://www.reddit.com/user/dpatrick86.json with after parameter: t1_cn3hqrd


In [45]:
# let's preview the first few results
reddit_data[0:4]

[{'username': 'dpatrick86',
  'kind': 't1',
  'created_utc': 1611261276.0,
  'subreddit_name_prefixed': 'r/coldshowers',
  'body': 'Fun fact... Norepinephrine is also the mechanism by which cold browns fat. When you adapt to the cold, one of the key adaptations is that browning and you owe it to norepinephrine.'},
 {'username': 'dpatrick86',
  'kind': 't1',
  'created_utc': 1609876550.0,
  'subreddit_name_prefixed': 'r/StandUpComedy',
  'body': 'This was great.'},
 {'username': 'dpatrick86',
  'kind': 't3',
  'created_utc': 1606926205.0,
  'subreddit_name_prefixed': 'r/PlantBasedDiet'},
 {'username': 'dpatrick86',
  'kind': 't3',
  'created_utc': 1606858947.0,
  'subreddit_name_prefixed': 'r/vegan'}]

*Question 2*

In [46]:
moderators = get_mods('marketing')

all_data = []
for moderator in moderators:
    print('processing content for moderator ' + moderator)
    reddit_data = reddit_activity_updated(moderator, ["created_utc", "subreddit_name_prefixed", "body"], 10)
    for item in reddit_data:
        all_data.append(item)

processing content for moderator dpatrick86
processing https://www.reddit.com/user/dpatrick86.json with after parameter: None
processing https://www.reddit.com/user/dpatrick86.json with after parameter: t1_emyw3bq
processing https://www.reddit.com/user/dpatrick86.json with after parameter: t3_9kh4ep
processing https://www.reddit.com/user/dpatrick86.json with after parameter: t1_dr8udl2
processing https://www.reddit.com/user/dpatrick86.json with after parameter: t1_dmp4kas
processing https://www.reddit.com/user/dpatrick86.json with after parameter: t1_div1xez
processing https://www.reddit.com/user/dpatrick86.json with after parameter: t1_dfbo1pf
processing https://www.reddit.com/user/dpatrick86.json with after parameter: t1_d5doj15
processing https://www.reddit.com/user/dpatrick86.json with after parameter: t1_cxfwgmo
processing https://www.reddit.com/user/dpatrick86.json with after parameter: t1_cn3hqrd
processing content for moderator v022450781
processing https://www.reddit.com/user/

processing https://www.reddit.com/user/AptSeagull.json with after parameter: t1_g3yq3us


In [157]:
# preview the results
print(len(all_data))
all_data[0:5]

2250


[{'username': 'dpatrick86',
  'kind': 't3',
  'created_utc': 1606926205.0,
  'subreddit_name_prefixed': 'r/PlantBasedDiet'},
 {'username': 'dpatrick86',
  'kind': 't3',
  'created_utc': 1606858947.0,
  'subreddit_name_prefixed': 'r/vegan'},
 {'username': 'dpatrick86',
  'kind': 't1',
  'created_utc': 1605216860.0,
  'subreddit_name_prefixed': 'r/coldshowers',
  'body': "How many buckets of ice is that? What's your setup like in this picture?"},
 {'username': 'dpatrick86',
  'kind': 't1',
  'created_utc': 1598754652.0,
  'subreddit_name_prefixed': 'r/Coronavirus',
  'body': 'Add only what you want by confirming on a diff-by-diff basis...\n`git add -p`'},
 {'username': 'dpatrick86',
  'kind': 't3',
  'created_utc': 1598458424.0,
  'subreddit_name_prefixed': 'r/breastfeeding'}]

---

### 2.7 Exporting data to CSV

__Importance__

Alright, you've almost made it. We've accomplished a whole lot by now: we just wrote a tool that extracts user-level data, and appends all that data in a list of dictionaries stored in Python. But, how can we port the data to another software program (e.g., R, Excel)?

We need to convert the data to a Comma Separated Values (CSV) file.

More specifically, we'd like to have a file with five columns, containing:
- the username,
- the type of content (post vs. comment),
- the timestamp (readable for humans),
- the sub reddit name, and
- the body, if present.

To faciliate writing to a CSV file, we'll make use of the `csv` library.

__Let's try it out__

Here, we'll start with a code snippet to parse the `username`, content type (`kind`) and timestamp (`created_utc`) to a CSV file. Run the snippet and the next cells to see the result (it should create a new file `reddit_posts.csv` in your current working directory)! 

In [47]:
import csv 

with open("reddit_posts.csv", "w") as csv_file: # <<- this is the line with the "flag" see exercises below
    writer = csv.writer(csv_file, delimiter = ";")
    writer.writerow(["username", "kind", "created_utc"])
    for content in all_data:
        writer.writerow([content['username'], content['kind'], content['created_utc']])
print('done!')

done!


Let's preview the content of that file directly in Jupyter, using the `pandas` library.

In [198]:
import pandas as pd

df = pd.read_csv("reddit_posts.csv", sep=";")
# shows top 10 rows
df.head(10)

,username,kind,created_utc,subreddit_name_prefixed,date,time,body
0,dpatrick86,t3,1.606926e+09,r/PlantBasedDiet,2020-12-02,16:23,NaN
1,dpatrick86,t3,1.606859e+09,r/vegan,2020-12-01,21:42,NaN
2,dpatrick86,t1,1.605217e+09,r/coldshowers,2020-11-12,21:34,How many buckets of ice is that? What's your s...
3,dpatrick86,t1,1.598755e+09,r/Coronavirus,2020-08-30,02:30,Add only what you want by confirming on a diff...
4,dpatrick86,t3,1.598458e+09,r/breastfeeding,2020-08-26,16:13,NaN
5,dpatrick86,t1,1.593915e+09,r/rubyonrails,2020-07-05,02:14,Paypal is a joke.
6,dpatrick86,t1,1.593203e+09,r/ScientificNutrition,2020-06-26,20:31,IGF-1 signaling is reduced more dramatically i...
7,dpatrick86,t1,1.592748e+09,r/insaneparents,2020-06-21,13:59,Sounds like your Dad has some psychological is...
8,dpatrick86,t1,1.592674e+09,r/neuro,2020-06-20,17:34,https://pubmed.ncbi.nlm.nih.gov/20228681/
9,dpatrick86,t1,1.591498e+09,r/Lambda1VR,2020-06-07,02:50,"You have to install Sidequest, which gives you..."


Good job, so far. But the file is not ready yet, so let's work on the exercises by extending the code snippet above.

---

__Exercise 9__  
The `reddit_posts.csv` file now only includes 3 columns (`userame`, `kind`, `created_utc`). Please add the following columns as well: the subreddit name, the text of users' comments, and the timestamp converted to YYYY-MM-DD (date, e.g., 2021-01-15) and HH:MM (e.g., 08:00). 

In [ ]:
# your answer goes here!

__Solutions__

In [48]:
import csv 
import time

with open("reddit_posts.csv", "w") as csv_file: # <<- this is the line with the "flag"l see exercises below
    writer = csv.writer(csv_file, delimiter = ";")
    writer.writerow(["username", "kind", "created_utc", "subreddit_name_prefixed", "date", "time", "body"])
    for content in all_data:
        time_converted = time.gmtime(content['created_utc'])
        datestamp = time.strftime("%Y-%m-%d", time_converted)
        timestamp = time.strftime("%H:%M", time_converted)
        try:
            bodytext = content['body']
        except:
            bodytext = ''
        writer.writerow([content['username'], content['kind'], content['created_utc'], content['subreddit_name_prefixed'],
                        datestamp, timestamp, bodytext])
print('done!')

done!


In [49]:
import pandas as pd

df = pd.read_csv("reddit_posts.csv", sep=";")
# shows top 10 rows
df.head(10)

,username,kind,created_utc,subreddit_name_prefixed,date,time,body
0,dpatrick86,t1,1.611261e+09,r/coldshowers,2021-01-21,20:34,Fun fact... Norepinephrine is also the mechani...
1,dpatrick86,t1,1.609877e+09,r/StandUpComedy,2021-01-05,19:55,This was great.
2,dpatrick86,t3,1.606926e+09,r/PlantBasedDiet,2020-12-02,16:23,NaN
3,dpatrick86,t3,1.606859e+09,r/vegan,2020-12-01,21:42,NaN
4,dpatrick86,t1,1.605217e+09,r/coldshowers,2020-11-12,21:34,How many buckets of ice is that? What's your s...
5,dpatrick86,t1,1.598755e+09,r/Coronavirus,2020-08-30,02:30,Add only what you want by confirming on a diff...
6,dpatrick86,t3,1.598458e+09,r/breastfeeding,2020-08-26,16:13,NaN
7,dpatrick86,t1,1.593915e+09,r/rubyonrails,2020-07-05,02:14,Paypal is a joke.
8,dpatrick86,t1,1.593203e+09,r/ScientificNutrition,2020-06-26,20:31,IGF-1 signaling is reduced more dramatically i...
9,dpatrick86,t1,1.592748e+09,r/insaneparents,2020-06-21,13:59,Sounds like your Dad has some psychological is...


### 2.8 Wrap-up

Good job - you've made it!

After working on this set of exercises, you should be able to further explore the Reddit API on your own. Does `sixwaystop313` spend the most time in subreddits in which he gets the most upvotes? Did his posting behavior change over time? Are moderators more likely to be a premium Reddit user? Think what data is required to obtain such data, and then try to extract such data.

At the same time, realize that we have only scratched the surface of what's possible with APIs. Headers and pagination played a vital role in requests and were sufficient thus far. Yet, the majority of [API endpoints](https://www.reddit.com/dev/api/) require authentication (OAuth), which is a whole topic on its own.